In [3]:
import os
import threading
import zipfile
import traceback
import csv
import re
import codecs
import glob
from datetime import datetime, timedelta
import json

In [30]:
src = 'C:/Users/Suhan/Desktop/dev/python 스터디/서예진.zip'
museid = '01234'
dst_root = 'input/voice/서예진/'
dst_postfix_dir = 'python 서예진/'
src_names = 'input/voice/서예진.zip'
input_dir = 'input/json'

In [31]:
os.path.isfile(src)

True

In [32]:
class ZIP_Thread(threading.Thread):
    def __init__(self, src, museid, dst_root, dst_postfix_dir, src_names):
        threading.Thread.__init__(self, target=self.run)

        self.src = src
#         self.mongo = mongo
        self.museid = museid
        self.dst_root = dst_root # input/voice/**.zip
        self.dst_postfix_dir = dst_postfix_dir
        self.src_names = src_names
        self.result_json = None
        self.input_map = {}
        
    def write_json(path, data):
        with open(path, 'w') as f:
            json.dump(data, f, indent=4, sort_keys=True, ensure_ascii=False)
            
    def get_input_map(self):
        return self.input_map
        
        
    def go(self):
        tcount = 0
        meta_json = {}
        meta_json['muse_id'] = self.museid
        meta_json['records'] = []
        
        with zipfile.ZipFile(self.src) as zip:
            metafile = [ x for x in zip.namelist() if x and x.find('.csv') >= 0 ]
            if len(metafile) == 1:
                zip.extract(metafile[0], path=self.dst_root)
                csv_file_path = os.path.join(self.dst_root, metafile[0])

                try:
                    # utf-8로 읽기
                    print('utf-8')
                    with open(csv_file_path, 'r', encoding='utf-8') as meta_f:
                        reader = csv.reader(meta_f)
                        for line in reader:
                            iname = re.sub(codecs.BOM_UTF8.decode('utf-8'), '', line[0])
                            self.input_map[iname] = line[1].strip()

                except:
                    print('cp949')
                    try:
                        #cp949 형식으로 읽기
                        with open(csv_file_path,'r',encoding='cp949') as meta_f:
                            reader = csv.reader(meta_f)
                            for line in reader:
                                iname = re.sub(codecs.BOM_UTF8.decode('utf-8'), '', line[0])
                                self.input_map[iname] = line[1].strip().decode('utf-8')

                    except Exception as ex:
#                       
                        traceback.print_exc()
#                       

                if self.input_map:
                    zip.extractall(self.dst_root)

                    for (path, text) in self.input_map.items():
                        rpath = path
                        ipath = os.path.join(self.dst_root, path.lstrip('/'))
                        tcount += 1
                        

                        if not os.path.exists(ipath):
                            #ipath = os.path.join(dst_root, '**', '%s.*' % path.lstrip('/'))
                            ipath = os.path.join(self.dst_root, '%s.*' % path.lstrip('/'))

                            ipath_list = glob.glob(ipath, recursive=True)
                            if not ipath_list:
#                           
                                continue

#                            
                            rpath = os.path.relpath(ipath_list[0],self. dst_root)

                        meta_json['records'].append({
                            'file_url': os.path.join(self.dst_postfix_dir, rpath),
                            'text': text
                        })
                    if len(meta_json['records']) > 0:
                        self.result_json = os.path.join(input_dir, '%s_%s.json' % (self.museid, datetime.now().strftime('%y%m%d'))).replace('\\','/')
                        ZIP_Thread.write_json(self.result_json, meta_json)
            return meta_json['records'],self.result_json


In [33]:
preproc = ZIP_Thread(src, museid, dst_root, dst_postfix_dir, src_names)
x = preproc.go()
x

utf-8


([{'file_url': 'python 서예진/1.m4a',
   'text': '내가 아직 어리고 지금보다 훨씬 더 쉽게 남의 말에 곧잘 화를 내던 시절 아버지께서는 나에게 한 가지 충고를 해주셨다.'},
  {'file_url': 'python 서예진/2.m4a', 'text': '그 후로 나는 그 충고를 마음속으로 항상 되새기곤 했다.'},
  {'file_url': 'python 서예진/3.m4a',
   'text': '남 의 잘잘못을 따질 때는 언제든지 이 세상 사람들이 너처럼 좋은 환경에서 자라지 못했다는 것을 기억해야 한다.'},
  {'file_url': 'python 서예진/4.m4a',
   'text': '아버지께서는 더 이상 말씀하시지 않았지만, 우리는 말없는 가운데서도 서로의 뜻을 잘 이해할 수 있었다.'},
  {'file_url': 'python 서예진/5.m4a',
   'text': '나는 아버지의 그 충고 속에는 보다 더 깊은 뜻이 담겨 있다는 것을 알아차렸다.'},
  {'file_url': 'python 서예진/6.m4a',
   'text': '아버지의 그 충고로 말미암아 나는 모든 판단을 될 수 있으면 뒤로 미루는 태도를 취하곤 했다.'},
  {'file_url': 'python 서예진/7.m4a',
   'text': '그것 때문에 성가시게 치근덕거리는 몇몇 사람들로부터 어려움을 겪기도 했다.'},
  {'file_url': 'python 서예진/8.m4a',
   'text': '이러한 성격의 소유자들은 나와 같은 정상적인 사람이 나타나면 금방 눈치채고 가까이 오게 마련이다.'},
  {'file_url': 'python 서예진/9.m4a',
   'text': '그것은 내가 거친, 세상에 알려지지 않은 사람들의 은밀한 비애를 많이 알고 있었기 때문이었다.'},
  {'file_url': 'python 서예진/10.m4a',
   'text': '어떤 확실한 근거로 은밀한 이야기가 드러나려 할 때면 나는 잠을 자

In [34]:
len(preproc.get_input_map())

6357

In [325]:
def write_json(path, data):
    with open(path, 'w') as f:
        json.dump(data, f, indent=4, sort_keys=True, ensure_ascii=False,)    
        
        

input_map = {}
tcount = 0
meta_json = {}
meta_json['muse_id'] = museid
meta_json['records'] = []

# with zipfile.ZipFile(src) as zip:
zip = zipfile.ZipFile(src)
metafile = [ x for x in zip.namelist() if x and x.find('.csv') >= 0 ]
if len(metafile) == 1:
    zip.extract(metafile[0], path=dst_root)
    csv_file_path = os.path.join(dst_root, metafile[0])

    try:
        # utf-8로 읽기
        print('utf-8')
        with open(csv_file_path, 'r', encoding='utf-8') as meta_f:
            reader = csv.reader(meta_f)
            for line in reader:
                iname = re.sub(codecs.BOM_UTF8.decode('utf-8'), '', line[0])
                input_map[iname] = line[1].strip()

    except:
        print('cp949')
        try:
            #cp949 형식으로 읽기
            with open(csv_file_path,'r',encoding='cp949') as meta_f:
                reader = csv.reader(meta_f)
                for line in reader:
                    iname = re.sub(codecs.BOM_UTF8.decode('utf-8'), '', line[0])
                    input_map[iname] = line[1].strip()

        except Exception as ex:
#                       
            traceback.print_exc()
#   
zip.close()

utf-8


In [326]:
input_map

{'1': '내가 아직 어리고 지금보다 훨씬 더 쉽게 남의 말에 곧잘 화를 내던 시절 아버지께서는 나에게 한 가지 충고를 해주셨다.',
 '2': '그 후로 나는 그 충고를 마음속으로 항상 되새기곤 했다.',
 '3': '남 의 잘잘못을 따질 때는 언제든지 이 세상 사람들이 너처럼 좋은 환경에서 자라지 못했다는 것을 기억해야 한다.',
 '4': '아버지께서는 더 이상 말씀하시지 않았지만, 우리는 말없는 가운데서도 서로의 뜻을 잘 이해할 수 있었다.',
 '5': '나는 아버지의 그 충고 속에는 보다 더 깊은 뜻이 담겨 있다는 것을 알아차렸다.',
 '6': '아버지의 그 충고로 말미암아 나는 모든 판단을 될 수 있으면 뒤로 미루는 태도를 취하곤 했다.',
 '7': '그것 때문에 성가시게 치근덕거리는 몇몇 사람들로부터 어려움을 겪기도 했다.',
 '8': '이러한 성격의 소유자들은 나와 같은 정상적인 사람이 나타나면 금방 눈치채고 가까이 오게 마련이다.',
 '9': '그것은 내가 거친, 세상에 알려지지 않은 사람들의 은밀한 비애를 많이 알고 있었기 때문이었다.',
 '10': '어떤 확실한 근거로 은밀한 이야기가 드러나려 할 때면 나는 잠을 자는 척하거나, 적의에 찬 경박한 짓을 하곤 했다.',
 '11': '사실 젊은이들의 은밀한 이야기는, 적어도 그것을 표현할 때 쓰는 말투도 대개 남의 말을 표절하게 마련이다.',
 '12': '때로는 어떤 의견 표명을 삼가고 있다는 것이 상대에게 무한한 희망을 줄 때도 있는 법이다.',
 '13': '이처럼 나의 너그러움을 한껏 과시하고 난 후에야 나는 너그러움에도 한계가 있다는 것을 알게 되었다.',
 '14': '어떤 한계만 넘기게 되면 나는 그 행위가 어떤 곳을 기반으로 해서 겉으로 드러나게 된 것이든 상관하지 않는다.',
 '15': '작년 가을, 동부에서 돌아왔을 때 나는 사람들이 똑같은 차림을 하고 일종의 도덕적인 자세를 취해 주었으면 하고 바랐다.',
 '16': '무슨 특권이라도 지닌 듯이 사람의 마음속

In [327]:
zip = zipfile.ZipFile(src)

if input_map:
    zip.extractall(dst_root)
    zip.close()

In [328]:
for (path, text) in input_map.items():
    rpath = path
    ipath = os.path.join(dst_root, path.lstrip('/'))
    tcount += 1


    if not os.path.exists(ipath):
        #ipath = os.path.join(dst_root, '**', '%s.*' % path.lstrip('/'))
        ipath = os.path.join(dst_root, '%s.*' % path.lstrip('/'))

        ipath_list = glob.glob(ipath, recursive=True) #오래걸리는 주범
        if not ipath_list:
#                           
            continue

#                            
        rpath = os.path.relpath(ipath_list[0], dst_root)

    meta_json['records'].append({
        'file_url': os.path.join(dst_postfix_dir, rpath),
        'text': text
    })


In [329]:
meta_json

{'muse_id': '0123',
 'records': [{'file_url': 'python 스터디/2',
   'text': '그 후로 나는 그 충고를 마음속으로 항상 되새기곤 했다.'},
  {'file_url': 'python 스터디/3',
   'text': '남 의 잘잘못을 따질 때는 언제든지 이 세상 사람들이 너처럼 좋은 환경에서 자라지 못했다는 것을 기억해야 한다.'},
  {'file_url': 'python 스터디/4',
   'text': '아버지께서는 더 이상 말씀하시지 않았지만, 우리는 말없는 가운데서도 서로의 뜻을 잘 이해할 수 있었다.'},
  {'file_url': 'python 스터디/5',
   'text': '나는 아버지의 그 충고 속에는 보다 더 깊은 뜻이 담겨 있다는 것을 알아차렸다.'},
  {'file_url': 'python 스터디/6',
   'text': '아버지의 그 충고로 말미암아 나는 모든 판단을 될 수 있으면 뒤로 미루는 태도를 취하곤 했다.'},
  {'file_url': 'python 스터디/7',
   'text': '그것 때문에 성가시게 치근덕거리는 몇몇 사람들로부터 어려움을 겪기도 했다.'},
  {'file_url': 'python 스터디/8',
   'text': '이러한 성격의 소유자들은 나와 같은 정상적인 사람이 나타나면 금방 눈치채고 가까이 오게 마련이다.'},
  {'file_url': 'python 스터디/9',
   'text': '그것은 내가 거친, 세상에 알려지지 않은 사람들의 은밀한 비애를 많이 알고 있었기 때문이었다.'},
  {'file_url': 'python 스터디/10',
   'text': '어떤 확실한 근거로 은밀한 이야기가 드러나려 할 때면 나는 잠을 자는 척하거나, 적의에 찬 경박한 짓을 하곤 했다.'},
  {'file_url': 'python 스터디/11',
   'text': '사실 젊은이들의 은밀한 이야기는, 적어도 그것을 표현할 때 쓰는 말투도 대개 남

In [330]:
if len(meta_json['records']) > 0:
    result_json = os.path.join(input_dir, '%s_%s.json' % (museid, datetime.now().strftime('%y%m%d'))).replace('\\','/')
    ZIP_Thread.write_json(result_json, meta_json)

UnicodeEncodeError: 'cp949' codec can't encode character '\xa0' in position 10: illegal multibyte sequence

In [315]:
input_zip_list = glob.glob('./*.zip', recursive=False) # input/voice 에있는 zip 찾기

ls = []
for i in input_zip_list:
    ls.append(i.replace('\\','/'))
    
zip_name = ''
number = -1
museid = ''

for idx, input_zip in enumerate(ls): # 압축파일마다 락이 걸려있는지 확인하기
    zip_name = os.path.splitext(os.path.relpath(input_zip, input_base_dir))[0].split('/') # [***]
    museid = zip_name[-1] # ***

    recordnow = datetime.now()
    if recordnow:
#         musestatus = mongo.find_status(id=museid) # DB에서 Collection 검색 
#         if musestatus: # Collection 존재한다
            pick_muse = mongo.lock_status_import(museid, instance_id, hb_seconds) # 락걸기 lock_status_import 함수 만들어야함
            if pick_muse:
                number = idx
                break

        else: # Collection 존재하지 않는다
            result = mongo.insert_status(museid, None, recordnow, 0) # collection을 만들어준다.

            pick_muse = mongo.lock_status_import(museid, instance_id, hb_seconds) # lock 걸기
            if pick_muse:
                number = idx
                break

NameError: name 'mongo' is not defined